In [1]:
# Define n
n = 4  # You can change this to experiment with other sizes
cell_size = 2 ** n
max_value = 2 ** n - 1  # since range is 0 to 2^(n-1) - 1

print(f"n = {n}")
print(f"Cell size (2^n): {cell_size}")
print(f"Max representable integer: {max_value}")

n = 4
Cell size (2^n): 16
Max representable integer: 15


In [2]:
# Define the function to convert int to binary cell representation
def int_to_cell_bits(value, n):
    """Convert an integer to binary string of length n."""
    if not (0 <= value < 2 ** n):
        raise ValueError(f"Value must be in range [0, {2**n - 1}]")
    
    return format(value, f'0{n}b')

In [3]:
# Now test for all values that fit in n bits
for i in range(2 ** n):
    print(f"{i} => {int_to_cell_bits(i, n)}")

0 => 0000
1 => 0001
2 => 0010
3 => 0011
4 => 0100
5 => 0101
6 => 0110
7 => 0111
8 => 1000
9 => 1001
10 => 1010
11 => 1011
12 => 1100
13 => 1101
14 => 1110
15 => 1111


In [4]:
def visualize_cell(value, n):
    binary = int_to_cell_bits(value, n)
    print("Cell Bits:")
    print(" | ".join(binary))

# Example
visualize_cell(5, n)

Cell Bits:
0 | 1 | 0 | 1


In [5]:
def cell_bits_to_int(bits):
    return int(bits, 2)

# Example
binary_str = int_to_cell_bits(9, n)
print(f"Bits: {binary_str} => Int: {cell_bits_to_int(binary_str)}")

Bits: 1001 => Int: 9


In [6]:
import random

# Parameters
n = 2
iterations = 10000
max_value = 2 ** n - 1  # full range [0, 2^n - 1]

def int_to_bits(value, n):
    return list(format(value, f'0{n}b'))  # returns list of chars: ['0','1',...]

def bits_to_int(bits):
    return int("".join(bits), 2)

def flip_random_bit(bits):
    index = random.randint(0, len(bits) - 1)
    bits[index] = '1' if bits[index] == '0' else '0'
    return bits

# Run experiment
errors = []

for _ in range(iterations):
    original = random.randint(0, max_value)
    bits = int_to_bits(original, n)
    flipped_bits = flip_random_bit(bits.copy())
    corrupted = bits_to_int(flipped_bits)
    squared_error = (original - corrupted) ** 2
    errors.append(squared_error)

# Calculate MSE
mse = sum(errors) / len(errors)
print(f"Mean Squared Error (MSE) over {iterations} iterations: {mse:.3f}")

Mean Squared Error (MSE) over 10000 iterations: 2.494


In [7]:
import itertools
import random
import os

def generate_bitstrings(n):
    return [format(i, f'0{n}b') for i in range(2**n)]

def flip_each_bit(bits):
    flipped = []
    for i in range(len(bits)):
        new_bits = list(bits)
        new_bits[i] = '1' if new_bits[i] == '0' else '0'
        flipped.append(''.join(new_bits))
    return flipped

def mse_for_mapping(mapping):
    mse = 0
    total = 0
    for bits, original_value in mapping.items():
        flipped_versions = flip_each_bit(bits)
        for corrupted_bits in flipped_versions:
            if corrupted_bits in mapping:
                corrupted_value = mapping[corrupted_bits]
                err = (original_value - corrupted_value) ** 2
                mse += err
                total += 1
            else:
                # Optionally: penalize undecodeable flips as max error
                # mse += (original_value - 0) ** 2
                # total += 1
                pass
    return mse / total if total > 0 else float('inf')

def gray_code(n):
    """Generate n-bit Gray code list of binary strings"""
    if n == 0:
        return ['']
    prev = gray_code(n - 1)
    return ['0' + code for code in prev] + ['1' + code for code in reversed(prev)]

def get_gray_code_mapping(n):
    gray_list = gray_code(n)
    mapping = {code: i for i, code in enumerate(gray_list)}
    return mapping


def run_for_n(n, log_file, sample_size=100000):
    bitstrings = generate_bitstrings(n)
    num_values = 2 ** n
    results = []

    if n < 4:
        all_perms = list(itertools.permutations(range(num_values)))
        for perm in all_perms:
            mapping = dict(zip(bitstrings, perm))
            mse = mse_for_mapping(mapping)
            results.append((mapping, mse))
    else:
        # Default mapping: binary string → int
        default_mapping = {format(i, f'0{n}b'): i for i in range(num_values)}
        mse_default = mse_for_mapping(default_mapping)
        results.append((default_mapping, mse_default))

        # Gray code mapping: gray_code[i] → i
        gray_list = gray_code(n)
        gray_mapping = {gray_list[i]: i for i in range(num_values)}
        mse_gray = mse_for_mapping(gray_mapping)
        results.append((gray_mapping, mse_gray))

        # Add random permutations
        for _ in range(sample_size):
            perm = random.sample(range(num_values), num_values)
            mapping = dict(zip(bitstrings, perm))
            mse = mse_for_mapping(mapping)
            results.append((mapping, mse))

    # Sort by MSE
    results.sort(key=lambda x: x[1])

    log_file.write(f"==== n = {n} ====\n")
    log_file.write("MSE mappings:\n")
    for m, e in results:
        log_file.write(f"MSE = {e:.2f}, Mapping: {m}\n")

    log_file.write("\n" + "="*40 + "\n\n")

mapping = get_gray_code_mapping(4)
print("Mapping:", mapping)

mapping = get_gray_code_mapping(5)
print("Mapping:", mapping)

# Log to file
print("Current working directory:", os.getcwd())

with open("bit_flip_mse_log.txt", "w") as f:
    for n in range(1, 6):
        run_for_n(n, f, sample_size=100000)

print("✅ Done. Results written to 'bit_flip_mse_log.txt'")





Mapping: {'0000': 0, '0001': 1, '0011': 2, '0010': 3, '0110': 4, '0111': 5, '0101': 6, '0100': 7, '1100': 8, '1101': 9, '1111': 10, '1110': 11, '1010': 12, '1011': 13, '1001': 14, '1000': 15}
Mapping: {'00000': 0, '00001': 1, '00011': 2, '00010': 3, '00110': 4, '00111': 5, '00101': 6, '00100': 7, '01100': 8, '01101': 9, '01111': 10, '01110': 11, '01010': 12, '01011': 13, '01001': 14, '01000': 15, '11000': 16, '11001': 17, '11011': 18, '11010': 19, '11110': 20, '11111': 21, '11101': 22, '11100': 23, '10100': 24, '10101': 25, '10111': 26, '10110': 27, '10010': 28, '10011': 29, '10001': 30, '10000': 31}
Current working directory: /home/ohad.eitan


✅ Done. Results written to 'bit_flip_mse_log.txt'


In [8]:
import itertools
import os

def generate_bitstrings(n):
    return [format(i, f'0{n}b') for i in range(2**n)]

def flip_each_bit(bits):
    flipped = []
    for i in range(len(bits)):
        new_bits = list(bits)
        new_bits[i] = '1' if new_bits[i] == '0' else '0'
        flipped.append(''.join(new_bits))
    return flipped

def gray_code(n):
    """Generate n-bit Gray code list of binary strings"""
    if n == 0:
        return ['']
    prev = gray_code(n - 1)
    return ['0' + code for code in prev] + ['1' + code for code in reversed(prev)]

def get_gray_code_mapping(n):
    gray_list = gray_code(n)
    mapping = {code: i for i, code in enumerate(gray_list)}
    return mapping

def mae_for_mapping(mapping):
    # Invert the mapping: bitstring → value
    inv_mapping = mapping
    total = 0
    mae = 0

    for bits, value in inv_mapping.items():
        flipped_versions = flip_each_bit(bits)
        for corrupted in flipped_versions:
            if corrupted in inv_mapping:
                corrupted_value = inv_mapping[corrupted]
                mae += abs(value - corrupted_value)
                total += 1
            else:
                # Optional: penalize invalid bitstrings (not in map)
                # mae += abs(value - 0)
                # total += 1
                pass

    return mae / total if total else float('inf')

def run_for_n(n, log_file, sample_size=100000):
    bitstrings = generate_bitstrings(n)
    num_values = 2 ** n
    results = []

    if n < 4:
        all_perms = list(itertools.permutations(range(num_values)))
        for perm in all_perms:
            mapping = dict(zip(bitstrings, perm))
            mae = mae_for_mapping(mapping)
            results.append((mapping, mae))
    

    else:
        # Always include the naive/default mapping: binary string → integer
        default_mapping = {format(i, f'0{n}b'): i for i in range(num_values)}
        mae_default = mae_for_mapping(default_mapping)
        results.append((default_mapping, mae_default))
    
        # Gray code mapping: gray_code[i] → i
        gray_list = gray_code(n)
        gray_mapping = {gray_list[i]: i for i in range(num_values)}
        mse_gray = mae_for_mapping(gray_mapping)
        results.append((gray_mapping, mse_gray))

        # Add random permutations
        for _ in range(sample_size):
            perm = random.sample(range(num_values), num_values)
            mapping = dict(zip(bitstrings, perm))
            mae = mae_for_mapping(mapping)
            results.append((mapping, mae))

    # Sort results by MAE
    results.sort(key=lambda x: x[1])

    log_file.write(f"==== n = {n} ====\n")
    log_file.write(f"MAE mappings:\n")
    for m, e in results:
        log_file.write(f"MAE = {e:.2f}, Mapping: {m}\n")

    log_file.write("\n" + "="*40 + "\n\n")

mapping = get_gray_code_mapping(4)
print("Mapping:", mapping)

mapping = get_gray_code_mapping(5)
print("Mapping:", mapping)


# Check working directory
print("Current working directory:", os.getcwd())

# Write to file
with open("bit_flip_mae_log.txt", "w") as f:
    for n in range(1, 6):  # Full for n = 1 to 3
        run_for_n(n, f, sample_size=100000)

print("✅ Done. Results written to 'bit_flip_mae_log.txt'")

Mapping: {'0000': 0, '0001': 1, '0011': 2, '0010': 3, '0110': 4, '0111': 5, '0101': 6, '0100': 7, '1100': 8, '1101': 9, '1111': 10, '1110': 11, '1010': 12, '1011': 13, '1001': 14, '1000': 15}
Mapping: {'00000': 0, '00001': 1, '00011': 2, '00010': 3, '00110': 4, '00111': 5, '00101': 6, '00100': 7, '01100': 8, '01101': 9, '01111': 10, '01110': 11, '01010': 12, '01011': 13, '01001': 14, '01000': 15, '11000': 16, '11001': 17, '11011': 18, '11010': 19, '11110': 20, '11111': 21, '11101': 22, '11100': 23, '10100': 24, '10101': 25, '10111': 26, '10110': 27, '10010': 28, '10011': 29, '10001': 30, '10000': 31}
Current working directory: /home/ohad.eitan
✅ Done. Results written to 'bit_flip_mae_log.txt'


In [9]:
import matplotlib.pyplot as plt

# === Main loop for plotting ===

def gray_to_int(g):
    b = int(g[0])
    result = b
    for i in range(1, len(g)):
        b ^= int(g[i])
        result = (result << 1) | b
    return result


# === Main loop for plotting ===
for n in range(1, 6):
    bitstrings = generate_bitstrings(n)
    num_values = 2 ** n

    if n < 4:
        all_perms = list(itertools.permutations(range(num_values)))
    else:
        # Sample 1000 random permutations
        all_perms = [random.sample(range(num_values), num_values) for _ in range(1000)]

        # Add naive binary mapping
        naive_perm = list(range(num_values))
        all_perms.append(naive_perm)

        # Add Gray code mapping
        gray_list = gray_code(n)
        gray_perm = [gray_to_int(g) for g in gray_list]
        all_perms.append(gray_perm)

    # Compute MSE for each mapping
    mse_values = []
    for perm in all_perms:
        mapping = dict(zip(bitstrings, perm))
        mse = mse_for_mapping(mapping)
        mse_values.append(mse)

    # Sort MSE values before plotting
    mse_values.sort()

    # Plot sorted MSE with connecting line
    plt.figure(figsize=(8, 5))
    plt.plot(range(len(mse_values)), mse_values, marker='o', linestyle='-', markersize=2)
    plt.title(f"Sorted MSE vs Mapping Index for n = {n}")
    plt.xlabel("Sorted Mapping Index")
    plt.ylabel("MSE")
    plt.grid(True)

    # Save image
    filename = f"mse_sorted_mapping_n{n}.png"
    plt.savefig(filename)
    plt.close()

    print(f"✅ Saved: {filename}")


✅ Saved: mse_sorted_mapping_n1.png
✅ Saved: mse_sorted_mapping_n2.png
✅ Saved: mse_sorted_mapping_n3.png
✅ Saved: mse_sorted_mapping_n4.png
✅ Saved: mse_sorted_mapping_n5.png


In [10]:
import matplotlib.pyplot as plt

# === Main loop for plotting ===

def gray_to_int(g):
    b = int(g[0])
    result = b
    for i in range(1, len(g)):
        b ^= int(g[i])
        result = (result << 1) | b
    return result


# === Main loop for plotting ===
for n in range(1, 6):
    bitstrings = generate_bitstrings(n)
    num_values = 2 ** n

    if n < 4:
        all_perms = list(itertools.permutations(range(num_values)))
    else:
        # Sample 1000 random permutations
        all_perms = [random.sample(range(num_values), num_values) for _ in range(1000)]

        # Add naive binary mapping
        naive_perm = list(range(num_values))
        all_perms.append(naive_perm)

        # Add Gray code mapping
        gray_list = gray_code(n)
        gray_perm = [gray_to_int(g) for g in gray_list]
        all_perms.append(gray_perm)

    # Compute MAE for each mapping
    mae_values = []
    for perm in all_perms:
        mapping = dict(zip(bitstrings, perm))
        mae = mae_for_mapping(mapping)
        mae_values.append(mae)

    # Sort MAE values before plotting
    mae_values.sort()

    # Plot sorted MAE with connecting line
    plt.figure(figsize=(8, 5))
    plt.plot(range(len(mae_values)), mae_values, marker='o', linestyle='-', markersize=2)
    plt.title(f"Sorted MAE vs Mapping Index for n = {n}")
    plt.xlabel("Sorted Mapping Index")
    plt.ylabel("MAE")
    plt.grid(True)

    # Save image
    filename = f"mae_sorted_mapping_n{n}.png"
    plt.savefig(filename)
    plt.close()

    print(f"✅ Saved: {filename}")


✅ Saved: mae_sorted_mapping_n1.png


✅ Saved: mae_sorted_mapping_n2.png
✅ Saved: mae_sorted_mapping_n3.png
✅ Saved: mae_sorted_mapping_n4.png
✅ Saved: mae_sorted_mapping_n5.png
